### References
- Finetune Llama 3 for Sentiment Analysis (https://www.kaggle.com/code/lucamassaron/fine-tune-llama-3-for-sentiment-analysis)
- Finetune Llama 2 for Sentiment Analysis (https://www.kaggle.com/code/lucamassaron/fine-tune-llama-2-for-sentiment-analysis)

In [1]:
!nvidia-smi

Thu Oct 24 03:38:27 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.90.07              Driver Version: 550.90.07      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla P100-PCIE-16GB           Off |   00000000:00:04.0 Off |                    0 |
| N/A   34C    P0             26W /  250W |       0MiB /  16384MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

# Libraries

In [2]:
# %pip install -q -U torch --index-url https://download.pytorch.org/whl/cu117
%pip install -q -U -i https://pypi.org/simple/ bitsandbytes
%pip install -q -U transformers
%pip install -q -U accelerate
%pip install -q -U datasets
%pip install -q -U trl
%pip install -q -U peft
# %pip install -q -U tensorboard

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [74]:
import os
import random
import numpy as np
import pandas as pd
import torch
import transformers
from datasets import load_dataset, Dataset
from transformers import BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer, pipeline
from sklearn.metrics import f1_score

# Config

In [4]:
seed = 42
lang = 'eng'
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu') # Might not work on Kaggle
model_id = 'meta-llama/Llama-3.2-1B-Instruct'

Disabling two features in PyTorch related to memory efficiency and speed during operations on the Graphics Processing Unit (GPU) specifically for the scaled dot product attention (SDPA) function.

In [5]:
torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)

In [6]:
def set_seed(seed):
    # Set random seed for NumPy
    np.random.seed(seed)

    # Set random seed for Torch
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if using multi-GPU
    torch.backends.cudnn.deterministic = True  # Ensures deterministic results
    torch.backends.cudnn.benchmark = False  # Avoids non-deterministic algorithms

    # Set random seed for Transformers
    transformers.set_seed(seed)

    # Optionally set random seed for sklearn and Python's own random module
    random.seed(seed)

    # Set random seed for os
    os.environ['PYTHONHASHSEED'] = str(seed)

    print(f"Random seed set to: {seed}")

set_seed(seed)

Random seed set to: 42


In [7]:
if not os.path.exists('/root/.cache/huggingface/token'):
    print("Hugging Face token (https://huggingface.co/settings/tokens):")
    hf_token = input()
    !huggingface-cli login --token $hf_token
else:
    print("Hugging Face token has already been saved")

Hugging Face token (https://huggingface.co/settings/tokens):


/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


# Data

## Load Data

In [54]:
data_files = {
    'train': f'preprocessed_data/train/{lang}.csv', 
    'val': f'preprocessed_data/val/{lang}.csv',
    'test': f'preprocessed_data/test/{lang}.csv',
}
dataset = load_dataset('alxxtexxr/SemEval2025-Task11-Dataset', data_files=data_files)

splits = data_files.keys()
df = {split: pd.DataFrame(dataset[split]) for split in splits}

cols = list(df['train'].columns)
print("DF columns:", cols)

emotion_cols = [col for col in cols if col not in ['Unnamed: 0', 'text', 'emotion']]
# neutral_emotion = df['train'][df['train'][emotion_cols].sum(axis=1) == 0]['emotion'].iloc[0]
# emotions = emotion_cols + [neutral_emotion]
print("Emotions columns:", emotion_cols)
print()

print("Train DF size:", len(df['train']))
print("Validation DF size:", len(df['val']))
print("Testing DF size:", len(df['test']))

DF columns: ['Unnamed: 0', 'text', 'emotion', 'anger', 'fear', 'joy', 'sad', 'surprise']
Emotions columns: ['anger', 'fear', 'joy', 'sad', 'surprise']

Train DF size: 2214
Validation DF size: 554
Testing DF size: 116


## Create Prompt Data

In [55]:
prompt_template = """
### Instruction
Detect the emotion(s) in the given input text. 
The detected emotion(s) can be one or a combination of the following: anger, fear, joy, sad, surprise, or neutral

### Input
Text: {text}

### Output
Emotion(s): {emotion}"""

def create_prompt(row):
    emotion_list = row['emotion'].replace(" ", "").split(",")
    emotion = ", ".join([f"{e}" for e in row['emotion'].replace(" ", "").split(",")])#[1:]#+ "]"
    # emotion = '\n'.join([f"- {e}" for e in emotion_list])[2:]
    return prompt_template.format(text=row['text'], emotion=emotion).strip()

def create_test_prompt(row):
    return prompt_template.format(text=row['text'], emotion="").strip()

df['train']['prompt'] = df['train'].apply(create_prompt, axis=1)
df['val']['prompt'] = df['val'].apply(create_test_prompt, axis=1)
df['test']['prompt'] = df['test'].apply(create_test_prompt, axis=1)

print("Train prompts:\n")
for prompt in df['train']['prompt'].head(3):
    print(prompt)
    print("================================================================================================================================================================================================")
print()
print("Testing prompts:\n")
for prompt in df['test']['prompt'].head(3):
    print(prompt)
    print("================================================================================================================================================================================================")

Train prompts:

### Instruction
Detect the emotion(s) in the given input text. 
The detected emotion(s) can be one or a combination of the following: anger, fear, joy, sad, surprise, or neutral

### Input
Text: I now have 12 of those canker sore suckers in my mouth along with a fever since friday.

### Output
Emotion(s): fear, sad
### Instruction
Detect the emotion(s) in the given input text. 
The detected emotion(s) can be one or a combination of the following: anger, fear, joy, sad, surprise, or neutral

### Input
Text: It just... went away.

### Output
Emotion(s): fear, sad, surprise
### Instruction
Detect the emotion(s) in the given input text. 
The detected emotion(s) can be one or a combination of the following: anger, fear, joy, sad, surprise, or neutral

### Input
Text: I naively walked up and stuck my head in the driver's window hole.

### Output
Emotion(s): fear, surprise

Testing prompts:

### Instruction
Detect the emotion(s) in the given input text. 
The detected emotion(s

In [56]:
max_seq_lengths = {split: df[split]['prompt'].str.len().max() for split in splits}
max_seq_length = max(max_seq_lengths.values())

print("Train max. prompt length:", max_seq_lengths['train'])
print("Validation max. prompt length:", max_seq_lengths['val'])
print("Testing max. prompt length:", max_seq_lengths['test'])
print()
print("Max. prompt length:", max_seq_length)

Train max. prompt length: 673
Validation max. prompt length: 586
Testing max. prompt length: 527

Max. prompt length: 673


## Create Hugging Face Dataset

In [57]:
datasets = {split: Dataset.from_pandas(df[split][['prompt']]) for split in ['train', 'val']}
datasets

{'train': Dataset({
     features: ['prompt'],
     num_rows: 2214
 }),
 'val': Dataset({
     features: ['prompt'],
     num_rows: 554
 })}

# Model

In [13]:
compute_dtype = torch.float16

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype=compute_dtype,
)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map=device,
    torch_dtype=compute_dtype,
    quantization_config=bnb_config,
)
model.config.use_cache = False
model.config.pretraining_tp = 1

tokenizer = AutoTokenizer.from_pretrained(model_id, max_seq_length=max_seq_length)
tokenizer.pad_token_id = tokenizer.eos_token_id

config.json:   0%|          | 0.00/877 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [14]:
# max_new_tokens = df['train'].apply(lambda row: len(tokenizer.encode(row['emotion'])), axis=1).max()
# print("Max. emotion tokens:", max_new_tokens)

# Evaluation without Finetuning 

In [72]:
y_true = df['val'].apply(lambda row: row[emotion_cols].tolist(), axis=1).tolist()

print(f"True Y ({len(y_true)}):")
y_true[:10]

True Y (554):


[[0, 1, 0, 1, 0],
 [0, 1, 0, 0, 1],
 [0, 0, 0, 1, 1],
 [0, 0, 0, 1, 0],
 [1, 1, 0, 0, 1],
 [0, 1, 0, 0, 0],
 [0, 1, 0, 0, 0],
 [0, 1, 0, 0, 1],
 [0, 1, 0, 1, 1],
 [1, 1, 0, 0, 0]]

In [73]:
def one_hot_encode_emotion(emotion, emotion_cols):
    emotions = emotion.replace(" ", "").split(",")
    one_hot_emotion = [1 if emotion_col in emotions else 0 for emotion_col in emotion_cols]
    return one_hot_emotion

def predict(df_, model, tokenizer):
    y_pred = []
    for i in range(len(df_)):
        prompt = df_.iloc[i]['prompt']
        pipe = pipeline(
            task='text-generation',
            model=model, 
            tokenizer=tokenizer,
            max_new_tokens=32,
            temperature=0.001,
        )
        result = pipe(prompt)
        pred_emotion = result[0]['generated_text'].replace(prompt, "").split("\n")[0].lower()
        pred_emotion_one_hot = one_hot_encode_emotion(pred_emotion, emotion_cols)
        y_pred.append(pred_emotion_one_hot)
        # if i == 5: break
    return y_pred

y_pred = predict(df['val'], model, tokenizer)
print(f"Predicted Y ({len(y_pred)})")
y_pred[:10]
# 10m 32.6s

Predicted Y (554)


[[0, 1, 1, 0, 0],
 [0, 1, 1, 0, 0],
 [0, 0, 0, 1, 0],
 [0, 0, 1, 0, 0],
 [1, 1, 0, 0, 0],
 [1, 1, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0],
 [1, 1, 1, 1, 1]]

In [75]:
# Calculate F1 score for each type of averaging method
f1_micro = f1_score(y_true, y_pred, average='micro', zero_division=0.0)
f1_macro = f1_score(y_true, y_pred, average='macro', zero_division=0.0)
# f1_weighted = f1_score(y_true, y_pred, average='weighted', zero_division=0.0)
# f1_samples = f1_score(y_true, y_pred, average='samples', zero_division=0.0)
f1_per_label = f1_score(y_true, y_pred, average=None, zero_division=0.0)

print(f'F1 Score (Micro-Average): {f1_micro}')
print(f'F1 Score (Macro-Average): {f1_macro}')
# print(f'F1 Score (Weighted-Average): {f1_weighted}')
# print(f'F1 Score (Samples-Average): {f1_samples}')
print()

# F1 score per label
for label, f1 in zip(emotion_cols, f1_per_label):
    print(f"F1 Score for '{label}': {f1}")

F1 Score (Micro-Average): 0.49547218628719275
F1 Score (Macro-Average): 0.49731087609348473

F1 Score for 'anger': 0.5555555555555555
F1 Score for 'fear': 0.5239085239085239
F1 Score for 'joy': 0.5384615384615384
F1 Score for 'sad': 0.5886287625418061
F1 Score for 'surprise': 0.28
